# Import Libraries

In [1]:
import gzip
import os
import polars as pl
import pandas as pd
import requests
import shutil
import datetime as dt
import sys
import numpy as np
from pathlib import Path

# Download the Data

In [2]:
TCGA_PROJECTS = [
    'TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 
    'TCGA-COAD', 'TCGA-DLBC', 'TCGA-ESCA', 'TCGA-GBM', 'TCGA-HNSC', 
    'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP', 'TCGA-LAML', 'TCGA-LGG', 
    'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 
    'TCGA-PAAD', 'TCGA-PCPG', 'TCGA-PRAD', 'TCGA-READ', 'TCGA-SARC', 
    'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT', 'TCGA-THCA', 'TCGA-THYM', 
    'TCGA-UCEC', 'TCGA-UCS', 'TCGA-UVM'
]
OMICS = {
    'methylation450': 'DNA_Methylation_450',
    'methylation27': 'DNA_Methylation_27',
    'star_fpkm': 'Gene_Expression',
    'clinical': 'Clinical_Data',
    'mirna': 'miRNA_Data'
}

In [3]:
def download_tcga_data(project, data_type):
    directory = OMICS[data_type]
    tcga_download_endpoint = f'https://gdc-hub.s3.us-east-1.amazonaws.com/download/{project}.{data_type}.tsv.gz'
    output_file_path = f'UCSC_TCGA_Data/{directory}/{project}.gz'

    decompressed_file_path = output_file_path.replace('.gz', '')
    if os.path.exists(decompressed_file_path):
        print(f"File already exists: {decompressed_file_path}")
        return
    
    response = requests.get(tcga_download_endpoint, stream=True)

    if response.status_code == 200:
        print(f"Starting the download for project {project}")
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        with open(output_file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"Downloaded file saved to {output_file_path}")
        
        with gzip.open(output_file_path, 'rb') as f_in:
            with open(output_file_path.replace('.gz', ''), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        
        print(f"Decompressed file saved to {output_file_path.replace('.gz', '')}")

        os.remove(output_file_path)
        print(f"Removed compressed file: {output_file_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

In [4]:
for project in TCGA_PROJECTS:
    download_tcga_data(project, "methylation450")
    # download_tcga_data(project, "methylation27")
    download_tcga_data(project, "star_fpkm")
    download_tcga_data(project, "mirna")

Starting the download for project TCGA-ACC
Downloaded file saved to UCSC_TCGA_Data/DNA_Methylation_450/TCGA-ACC.gz
Decompressed file saved to UCSC_TCGA_Data/DNA_Methylation_450/TCGA-ACC
Removed compressed file: UCSC_TCGA_Data/DNA_Methylation_450/TCGA-ACC.gz
Starting the download for project TCGA-ACC
Downloaded file saved to UCSC_TCGA_Data/Gene_Expression/TCGA-ACC.gz
Decompressed file saved to UCSC_TCGA_Data/Gene_Expression/TCGA-ACC
Removed compressed file: UCSC_TCGA_Data/Gene_Expression/TCGA-ACC.gz
Starting the download for project TCGA-ACC
Downloaded file saved to UCSC_TCGA_Data/miRNA_Data/TCGA-ACC.gz
Decompressed file saved to UCSC_TCGA_Data/miRNA_Data/TCGA-ACC
Removed compressed file: UCSC_TCGA_Data/miRNA_Data/TCGA-ACC.gz
Starting the download for project TCGA-BLCA
Downloaded file saved to UCSC_TCGA_Data/DNA_Methylation_450/TCGA-BLCA.gz
Decompressed file saved to UCSC_TCGA_Data/DNA_Methylation_450/TCGA-BLCA
Removed compressed file: UCSC_TCGA_Data/DNA_Methylation_450/TCGA-BLCA.gz
Sta

## Join the projects for DNA Methylation 27k and 450k

In [3]:
def get_illumina27_patients(project: str):
    file_path = Path('UCSC_TCGA_Data').joinpath('DNA_Methylation_27', project)
    if not file_path.exists():
        return []
    else:
        df = pl.read_csv(file_path, separator='\t')
        return df.columns

def join_methylation_projects(project):
    output_path = os.path.join('UCSC_TCGA_Data', 'DNA_Methylation')
    output_file = Path(output_path).joinpath(project)

    # check whether the output path exists
    if not Path(output_path).exists():
        Path(output_path).mkdir(parents=True, exist_ok=True)

    # read and join the files
    file_path_27 = os.path.join('UCSC_TCGA_Data', 'DNA_Methylation_27', project)
    file_path_450 = os.path.join('UCSC_TCGA_Data', 'DNA_Methylation_450', project)

    # validate if the joining procedure shall be started
    if not (Path(file_path_27).exists() or Path(file_path_450).exists()):
        print(f'Both files do not exists to join. They might have been joined already or accidently been removed')
        return

    # check whether a 27k file could be downloaded
    if not Path(file_path_27).exists():
        print(f'No Illumina 27k file was available for project {project}, moving Illumina 450k experiment to {output_file} instead')
        Path(file_path_450).replace(output_file)
    else:
        print(f'Reading and joining both DNA Methylation files for project {project}')
        df_27 = pl.read_csv(file_path_27, separator='\t')
        df_450 = pl.read_csv(file_path_450, separator='\t')
    
        df = df_27.join(df_450, on='Composite Element REF', how='full', coalesce=True)
        print(f'Files joined from shapes {df_27.shape} and {df_450.shape} to shape {df.shape}')
        
        df.write_csv(output_file, separator='\t')
        print(f'Methylation files joined into {output_file}')
    
def remove_separate_dna_methylation_projects(project):
    file_path = Path('UCSC_TCGA_Data')
    print(f'Removing Illumina 27k and Illumina 450k files of {project}')
    file_path.joinpath('DNA_Methylation_27', project).unlink(missing_ok=True)
    file_path.joinpath('DNA_Methylation_450', project).unlink(missing_ok=True)

In [4]:
illumina_27_patients = []
for project in TCGA_PROJECTS:
    illumina_27_patients.extend(get_illumina27_patients(project))
    join_methylation_projects(project)
    remove_separate_dna_methylation_projects(project)
illumina_27_patients = [item for item in illumina_27_patients if item != 'Composite Element REF']

Both files do not exists to join. They might have been joined already or accidently been removed
Removing Illumina 27k and Illumina 450k files of TCGA-ACC
Both files do not exists to join. They might have been joined already or accidently been removed
Removing Illumina 27k and Illumina 450k files of TCGA-BLCA
Both files do not exists to join. They might have been joined already or accidently been removed
Removing Illumina 27k and Illumina 450k files of TCGA-BRCA
Both files do not exists to join. They might have been joined already or accidently been removed
Removing Illumina 27k and Illumina 450k files of TCGA-CESC
Both files do not exists to join. They might have been joined already or accidently been removed
Removing Illumina 27k and Illumina 450k files of TCGA-CHOL
Both files do not exists to join. They might have been joined already or accidently been removed
Removing Illumina 27k and Illumina 450k files of TCGA-COAD
Both files do not exists to join. They might have been joined alr

# Process the Data

## DNA Methylation

### Determine which probes to keep

In [5]:
def create_probes_to_remove_set():
    hg38_genes_file_path = 'humanmethylation450_15017482_v1-2.csv'
    df = pl.read_csv(hg38_genes_file_path, separator=',', null_values=['NA'], skip_lines=7, columns=['IlmnID', 'CHR', 'UCSC_RefGene_Name'])

    no_gene_associated = set()
    sexual_chr_probes = set()
    chrM_probes = set()
    non_cpg_probes = set()
    non_standard_transcript_probes = set()

    # standard_transcript_types = {"protein_coding", "lncRNA", "miRNA"}

    for row in df.iter_rows(named=True):
        chromosome = row['CHR']  
        probe_id = str(row['IlmnID']) 
        gene_id = str(row['UCSC_RefGene_Name'])
        gene_names = gene_id.split(";") 
        # transcript_types = row['transcriptTypes']
            
        if gene_id == 'None':
            no_gene_associated.add(probe_id)
            continue

        if not probe_id.startswith('cg'):
            non_cpg_probes.add(probe_id)
            continue

        if chromosome == 'chrY' or chromosome=='chrX':
            sexual_chr_probes.add(probe_id)
            continue

        if chromosome=='chrM':
            chrM_probes.add(probe_id)
            continue

        if len(gene_names) == 1: # Remove AL/AF lncRNA
            if '.' in gene_names[0]:
                non_standard_transcript_probes.add(probe_id)

    print(f"Number of probes without gene: {len(no_gene_associated)}")
    print(f"Number of non standard probes: {len(non_cpg_probes)}")
    print(f"Number of sexual chromosome probes: {len(sexual_chr_probes)}")
    print(f"Number of M chromosome probes: {len(chrM_probes)}")
    print(f"Number of non standard transcript type probes: {len(non_standard_transcript_probes)}")

    probes_to_remove = no_gene_associated.union(non_cpg_probes).union(sexual_chr_probes). \
        union(chrM_probes).union(non_standard_transcript_probes)
    probes_to_keep = {*df.select('IlmnID').filter(~df['IlmnID'].is_in(probes_to_remove)).to_series().to_list()}

    return probes_to_remove, probes_to_keep

In [6]:
probes_to_remove, probes_to_keep = create_probes_to_remove_set()

Number of probes without gene: 120568
Number of non standard probes: 1217
Number of sexual chromosome probes: 0
Number of M chromosome probes: 0
Number of non standard transcript type probes: 25


### Remove unwanted probes 

In [7]:
def is_valid_probe(probe_name):
    return probe_name not in probes_to_remove

def process_project_methylation_files(protocol, project):
    print(f'Processing data for project {project}')
    project_file_path = f'UCSC_TCGA_Data/{protocol}/{project}'
    output_file_path = (f'UCSC_TCGA_Data_Clean/{protocol}/'
        f'{project}_filtered_methylation.tsv')
    
    if os.path.exists(output_file_path):
        return

    if not os.path.exists(project_file_path):
        return

    df = pl.read_csv(project_file_path, separator='\t',
                     null_values=['NA'])
    print(f'Initial shape {df.shape}')
    df = df.rename({'Composite Element REF': 'Probe'}) 

    df_filtered = df.filter(~df['Probe'].is_in(probes_to_remove))

    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    print(df_filtered.shape)
    df_filtered.write_csv(output_file_path, separator='\t')
    print(f'Finished writing data to file {output_file_path}')

    # Remove the original file to save up space  
    #os.remove(project_file_path)
    #print(f"Deleted original file {project_file_path} to free up space.")

In [8]:
for project in TCGA_PROJECTS:
    process_project_methylation_files('DNA_Methylation_450', project)
    process_project_methylation_files('DNA_Methylation_27', project)
    process_project_methylation_files('DNA_Methylation', project)

Processing data for project TCGA-ACC
Processing data for project TCGA-ACC
Processing data for project TCGA-ACC
Processing data for project TCGA-BLCA
Processing data for project TCGA-BLCA
Processing data for project TCGA-BLCA
Processing data for project TCGA-BRCA
Processing data for project TCGA-BRCA
Processing data for project TCGA-BRCA
Processing data for project TCGA-CESC
Processing data for project TCGA-CESC
Processing data for project TCGA-CESC
Processing data for project TCGA-CHOL
Processing data for project TCGA-CHOL
Processing data for project TCGA-CHOL
Processing data for project TCGA-COAD
Processing data for project TCGA-COAD
Processing data for project TCGA-COAD
Processing data for project TCGA-DLBC
Processing data for project TCGA-DLBC
Processing data for project TCGA-DLBC
Processing data for project TCGA-ESCA
Processing data for project TCGA-ESCA
Processing data for project TCGA-ESCA
Processing data for project TCGA-GBM
Processing data for project TCGA-GBM
Processing data f

## Gene Expression and Copy Number Variation

### Determine which genes to keep

In [9]:
def create_gencode_df():
    human_gencode = 'gencode.v49.annotation.gtf'

    human_gencode_df = pl.read_csv(human_gencode, separator='\t',
                        null_values=['NA'], has_header=False, 
                        comment_prefix='#', truncate_ragged_lines=True)

    human_gencode_df = human_gencode_df.rename({
        "column_1": "chromosome",
        "column_2": "source",
        "column_3": "feature",
        "column_4": "start",
        "column_5": "end",
        "column_6": "score",
        "column_7": "strand",
        "column_8": "frame",
        "column_9": "attributes",
    })

    human_gencode_df = human_gencode_df.with_columns([
        human_gencode_df["attributes"].str.extract(r'gene_id "([^"]+)"', 1).alias("gene_id"),
        human_gencode_df["attributes"].str.extract(r'gene_type "([^"]+)"', 1).alias("gene_type"),
    ])
    human_gencode_df = human_gencode_df.replace_column(0,
        pl.Series('gene_id', human_gencode_df['gene_id'].str.extract(r"(\w+)"))
    )

    human_gencode_df = human_gencode_df.select(["gene_id", "gene_type"]).unique()
    return human_gencode_df

human_gencode_df = create_gencode_df()

In [10]:
def create_genes_to_remove_set(human_gencode_df):
    hg38_genes_file_path = 'gencode.v36.annotation.gtf.gene.probemap' 

    df = pl.read_csv(hg38_genes_file_path, separator='\t',
                     null_values=['NA'])

    selected_columns = df.select(['id', 'gene', 'chrom'])

    sexual_chr_genes = set()
    chrM_probes = set()
    non_standard_genes = set()
    standard_transcript_types = {"protein_coding", "lncRNA", "miRNA"}

    gencode_map = dict(zip(human_gencode_df['gene_name'].to_list(),
                           human_gencode_df['gene_type'].to_list()))

    for row in selected_columns.iter_rows(named=True):
        chromosome = row['chrom']  
        gene_id = str(row['id']) 
        gene_name = str(row['gene'])

        if chromosome == 'chrY' or chromosome == 'chrX':
            sexual_chr_genes.add(gene_id)
            continue

        if chromosome=='chrM':
            chrM_probes.add(gene_id)
            continue

        if gene_name in gencode_map.keys():
            gene_type = gencode_map[gene_name]
            if gene_type not in standard_transcript_types:
                non_standard_genes.add(gene_id)
                continue
        
        if '.' in gene_name:
            non_standard_genes.add(gene_id) # Remove pseudogenes and AL/AF lncRNA

    print(f"Number of non standard genes: {len(non_standard_genes)}")
    print(f"Number of sexual chromosome probes: {len(sexual_chr_genes)}")
    print(f"Number of M chromosome probes: {len(chrM_probes)}")

    ensemblids_to_remove = non_standard_genes.union(sexual_chr_genes).union(chrM_probes)

    return ensemblids_to_remove

In [11]:
ensemblids_to_remove = [] # create_genes_to_remove_set(human_gencode_df)

### Remove unwanted genes

In [12]:
def process_project_gene_level_files(project, type):
    print(f'Processing data for project {project}')
    if type == 'mirna':
        project_file_path = f'UCSC_TCGA_Data/miRNA_Data/{project}'
        output_file_path = ('UCSC_TCGA_Data_Clean/miRNA_Expression/'
            f'{project}_filtered_mirna_expression.tsv')
    elif type == 'gene_expression':
        project_file_path = f'UCSC_TCGA_Data/Gene_Expression/{project}'
        output_file_path = ('UCSC_TCGA_Data_Clean/Gene_Expression/'
            f'{project}_filtered_gene_expression.tsv')
    elif type == 'lncrna':
        project_file_path = f'UCSC_TCGA_Data/Gene_Expression/{project}'
        output_file_path = ('UCSC_TCGA_Data_Clean/lncRNA_Expression/'
            f'{project}_filtered_lncrna_expression.tsv')
    
    if os.path.exists(output_file_path):
        return

    df = pl.read_csv(project_file_path, separator='\t',
                     null_values=['NA'])
    print(f'Initial shape {df.shape}')  

    if type == 'mirna':
        df_filtered = df
    elif type == 'gene_expression':
        df = df.filter(~df['Ensembl_ID'].str.ends_with('_PAR_Y'))
        df = df.replace_column(0,
                pl.Series('Gene', df['Ensembl_ID'].str.extract(r"(\w+)"))
            )
        df_filtered = df.filter(~df['Gene'].is_in(human_gencode_df['gene_id'].filter(human_gencode_df['gene_type'].is_in(['lncRNA', 'miRNA'])))
                               )
    elif type == 'lncrna':
        df = df.filter(~df['Ensembl_ID'].str.ends_with('_PAR_Y'))
        df = df.replace_column(0,
                pl.Series('Gene', df['Ensembl_ID'].str.extract(r"(\w+)"))
            )
        df_filtered = df.filter(df['Gene'].is_in(human_gencode_df['gene_id'].filter(human_gencode_df['gene_type'].is_in(['lncRNA']))))
    
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    print(df_filtered.shape)
    df_filtered.write_csv(output_file_path, separator='\t')
    print(f'Finished writing data to file {output_file_path}')

In [13]:
for project in TCGA_PROJECTS:
    process_project_gene_level_files(project, "gene_expression")
    process_project_gene_level_files(project, "lncrna")    
    process_project_gene_level_files(project, "mirna")

Processing data for project TCGA-ACC
Processing data for project TCGA-ACC
Processing data for project TCGA-ACC
Processing data for project TCGA-BLCA
Processing data for project TCGA-BLCA
Processing data for project TCGA-BLCA
Processing data for project TCGA-BRCA
Processing data for project TCGA-BRCA
Processing data for project TCGA-BRCA
Processing data for project TCGA-CESC
Processing data for project TCGA-CESC
Processing data for project TCGA-CESC
Processing data for project TCGA-CHOL
Processing data for project TCGA-CHOL
Processing data for project TCGA-CHOL
Processing data for project TCGA-COAD
Processing data for project TCGA-COAD
Processing data for project TCGA-COAD
Processing data for project TCGA-DLBC
Processing data for project TCGA-DLBC
Processing data for project TCGA-DLBC
Processing data for project TCGA-ESCA
Processing data for project TCGA-ESCA
Processing data for project TCGA-ESCA
Processing data for project TCGA-GBM
Processing data for project TCGA-GBM
Processing data f

# Intersect the Samples Available for each Omic

In [14]:
def intersect_project_samples(project):
    print(f'Beginning intersection for {project}')  
    dna_methylation_path = f'UCSC_TCGA_Data_Clean/DNA_Methylation/{project}_filtered_methylation.tsv'
    gene_expression_path = f'UCSC_TCGA_Data_Clean/Gene_Expression/{project}_filtered_gene_expression.tsv'
    mirna_path = f'UCSC_TCGA_Data_Clean/miRNA_Expression/{project}_filtered_mirna_expression.tsv'
    lncrna_path = f'UCSC_TCGA_Data_Clean/lncRNA_Expression/{project}_filtered_lncrna_expression.tsv'

    dna_df = pl.read_csv(dna_methylation_path, separator='\t')
    ge_df = pl.read_csv(gene_expression_path, separator='\t')
    mirna_df = pl.read_csv(mirna_path, separator='\t')
    lncrna_df = pl.read_csv(lncrna_path, separator='\t')

    dna_sample_cols = dna_df.select(pl.col(pl.Float64)).columns 
    ge_sample_cols = ge_df.select(pl.col(pl.Float64)).columns  
    mirna_sample_cols = mirna_df.select(pl.col(pl.Float64)).columns
    lncrna_sample_cols = lncrna_df.select(pl.col(pl.Float64)).columns
    
    first_col_dna = dna_df.columns[0]
    first_col_ge = ge_df.columns[0]
    first_col_mirna = mirna_df.columns[0]
    first_col_lncrna = lncrna_df.columns[0]
    
    common_samples = list(set(dna_sample_cols) & set(ge_sample_cols) & set(mirna_sample_cols) & set(lncrna_sample_cols))
    
    dna_cols_to_keep = [first_col_dna] + common_samples
    ge_cols_to_keep = [first_col_ge] + common_samples
    mirna_cols_to_keep = [first_col_mirna] + common_samples
    lncrna_cols_to_keep = [first_col_lncrna] + common_samples

    dna_df_filtered = dna_df.select(dna_cols_to_keep)
    ge_df_filtered = ge_df.select(ge_cols_to_keep)
    mirna_df_filtered = mirna_df.select(mirna_cols_to_keep)
    lncrna_df_filtered = lncrna_df.select(lncrna_cols_to_keep)
    print(f'dna methylation shape: {dna_df_filtered.shape}')
    print(f'gene expression shape: {ge_df_filtered.shape}')
    print(f'mirna expression shape: {mirna_df_filtered.shape}')
    print(f'lncrna expression shape: {lncrna_df_filtered.shape}')

    dna_df_filtered.write_csv(dna_methylation_path, separator='\t')
    ge_df_filtered.write_csv(gene_expression_path, separator='\t')
    mirna_df_filtered.write_csv(mirna_path, separator='\t')
    lncrna_df_filtered.write_csv(lncrna_path, separator='\t')
    print(f'Finished intersecting matrices for project {project}')
    return dna_df_filtered.shape[1]

In [15]:
sample_number = 0
for project in TCGA_PROJECTS:
    count = intersect_project_samples(project)
    sample_number += int(count)
print(f'Total samples: {sample_number}')

Beginning intersection for TCGA-ACC
dna methylation shape: (365468, 80)
gene expression shape: (42964, 80)
mirna expression shape: (1881, 80)
lncrna expression shape: (15775, 80)
Finished intersecting matrices for project TCGA-ACC
Beginning intersection for TCGA-BLCA
dna methylation shape: (365468, 424)
gene expression shape: (42964, 424)
mirna expression shape: (1881, 424)
lncrna expression shape: (15775, 424)
Finished intersecting matrices for project TCGA-BLCA
Beginning intersection for TCGA-BRCA
dna methylation shape: (365468, 854)
gene expression shape: (42964, 854)
mirna expression shape: (1881, 854)
lncrna expression shape: (15775, 854)
Finished intersecting matrices for project TCGA-BRCA
Beginning intersection for TCGA-CESC
dna methylation shape: (365468, 308)
gene expression shape: (42964, 308)
mirna expression shape: (1881, 308)
lncrna expression shape: (15775, 308)
Finished intersecting matrices for project TCGA-CESC
Beginning intersection for TCGA-CHOL
dna methylation shape

# Verify Sample Type

### Remove samples that are not primary tumor

In [16]:
def remove_non_primary_tumor_samples(project):
    
    dna_methylation_path = f'UCSC_TCGA_Data_Clean/DNA_Methylation/{project}_filtered_methylation.tsv'
    gene_expression_path = f'UCSC_TCGA_Data_Clean/Gene_Expression/{project}_filtered_gene_expression.tsv'
    mirna_expression_path = f'UCSC_TCGA_Data_Clean/miRNA_Expression/{project}_filtered_mirna_expression.tsv'
    lncrna_expression_path = f'UCSC_TCGA_Data_Clean/lncRNA_Expression/{project}_filtered_lncrna_expression.tsv'

    dna_df = pl.read_csv(dna_methylation_path, separator='\t')
    ge_df = pl.read_csv(gene_expression_path, separator='\t')
    mirna_df = pl.read_csv(mirna_expression_path, separator='\t')
    lncrna_df = pl.read_csv(lncrna_expression_path, separator='\t')
    print(f'Initial number of samples {dna_df.shape[1]}')

    first_col_dna = dna_df.columns[0]
    first_col_ge = ge_df.columns[0]
    first_col_mirna = mirna_df.columns[0]
    first_col_lncrna = lncrna_df.columns[0]

    project_sample_types = {}

    columns_to_keep = []  
    dna_sample_cols = dna_df.select(pl.col(pl.Float64)).columns 
    for column in dna_sample_cols:
        sample_type = column.split('-')[3][:2] 
        if sample_type not in project_sample_types:
            project_sample_types[sample_type] = 1
        else:
            project_sample_types[sample_type] += 1
        if sample_type == '01' or sample_type == '11':   
            columns_to_keep.append(column)

    print(project_sample_types)

    dna_cols_to_keep = [first_col_dna] + columns_to_keep
    ge_cols_to_keep = [first_col_ge] + columns_to_keep
    mirna_cols_to_keep = [first_col_mirna] + columns_to_keep
    lncrna_cols_to_keep = [first_col_lncrna] + columns_to_keep

    dna_df_filtered = dna_df.select(dna_cols_to_keep)
    ge_df_filtered = ge_df.select(ge_cols_to_keep)
    mirna_df_filtered = mirna_df.select(mirna_cols_to_keep)
    lncrna_df_filtered = lncrna_df.select(lncrna_cols_to_keep)

    dna_df_filtered.write_csv(dna_methylation_path, separator='\t')
    ge_df_filtered.write_csv(gene_expression_path, separator='\t')
    mirna_df_filtered.write_csv(mirna_expression_path, separator='\t')
    lncrna_df_filtered.write_csv(lncrna_expression_path, separator='\t')
    print(f'Final number of samples {dna_df_filtered.shape[1]}')
    print(f'Finished verifying samples type for project {project}')
    return dna_df_filtered.shape[1]

In [17]:
sample_number = 0
for project in TCGA_PROJECTS:
    count = remove_non_primary_tumor_samples(project)
    sample_number += count
print(f'Total samples: {sample_number}')

Initial number of samples 80
{'01': 79}
Final number of samples 80
Finished verifying samples type for project TCGA-ACC
Initial number of samples 424
{'01': 406, '11': 17}
Final number of samples 424
Finished verifying samples type for project TCGA-BLCA
Initial number of samples 854
{'01': 777, '11': 76}
Final number of samples 854
Finished verifying samples type for project TCGA-BRCA
Initial number of samples 308
{'01': 304, '11': 3}
Final number of samples 308
Finished verifying samples type for project TCGA-CESC
Initial number of samples 45
{'11': 9, '01': 35}
Final number of samples 45
Finished verifying samples type for project TCGA-CHOL
Initial number of samples 306
{'01': 301, '11': 4}
Final number of samples 306
Finished verifying samples type for project TCGA-COAD
Initial number of samples 48
{'01': 47}
Final number of samples 48
Finished verifying samples type for project TCGA-DLBC
Initial number of samples 195
{'01': 183, '11': 11}
Final number of samples 195
Finished verify

# Replace NaN for null

In [18]:
def replace_all_nan():
    directories = ['DNA_Methylation', 'Gene_Expression', 'miRNA_Expression', 'lncRNA_Expression']
    for directory in directories:
        if not Path('UCSC_TCGA_Data_Clean').joinpath(directory).exists():
            continue
        for file_path in Path('UCSC_TCGA_Data_Clean').joinpath(directory).glob('**/*.tsv'):
            df = pl.read_csv(file_path, separator='\t')
            print((df.null_count().sum_horizontal() > 0).any())
            replaced_nan_df = df.fill_nan(None)
            print((replaced_nan_df.null_count().sum_horizontal() > 0).any())
            replaced_nan_df.write_csv(file_path, separator='\t')

In [19]:
replace_all_nan()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

# Aggregate DataFrames by omics and remove Missing Values

In [20]:
# assign the maximum number of features after the previously executed
# it is hard coded but can also be simply augmented dynamically by reading the file of the first project
MAX_FEATURE_NUMBER = {
    'miRNA_Expression': 1881,
    'lncRNA_Expression': 15786,
    'Gene_Expression': 42995
}

In [21]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [22]:
def get_all_patients(omics_type):
    print(f'{dt.datetime.now()}: Reading all patient ids')
    directory = f'UCSC_TCGA_Data_Clean/{omics_type}'
    columns = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        
        if not os.path.isdir(file_path):
            df = pl.read_csv(file_path, separator='\t', null_values=['NA'])
            columns.extend([*df.columns[1:]])
    print(f'{dt.datetime.now()}: All patient ids read, in total {len(columns)} patients')
    return columns

# drop the patients from the illumina 27 list which were filtered by the intersection method above
all_patients = get_all_patients('DNA_Methylation')
illumina_27_patients = [patient for patient in illumina_27_patients if patient in all_patients]

2026-01-15 12:28:26.214965: Reading all patient ids
2026-01-15 12:28:52.009226: All patient ids read, in total 8494 patients


In [23]:
def remove_missing_values_threshold(df, var): 
    numeric_cols = df.columns[1:] 
    num_columns = len(numeric_cols)
    na_threshold = num_columns * 0.05
    variance_threshold = var

    filtered_df = df.with_columns(
            pl.sum_horizontal([pl.col(c).is_null() for c in numeric_cols])
                .alias("na_count"),
            pl.concat_list([pl.col(c).cast(pl.Float64) for c in numeric_cols])
            .list.var()
            .alias("row_var")
        )
    if var != 0:
            filtered_df = filtered_df.filter(
                (pl.col("na_count") <= na_threshold) &
                (pl.col("row_var") >= variance_threshold) 
            ).drop(["na_count", "row_var"]) 
    else: 
        filtered_df = filtered_df.filter(
                (pl.col("na_count") <= na_threshold) &
                (pl.col("row_var") > variance_threshold)
            ).drop(["na_count", 'row_var'])

    print(filtered_df.shape)
    return filtered_df

In [24]:
def remove_missing_and_zero_expression(type, id_col, chunks):
    directory = f'UCSC_TCGA_Data_Clean/{type}'
    output_directory = f'UCSC_TCGA_Data_Clean/Aggregated/'
    output_file_path = os.path.join(output_directory, f'{type}.tsv')
    patients = [id_col]
    str_cols = {id_col}
    patients.extend(get_all_patients(type))
    out_df_cols = {col: [0.0] for col in patients if col not in str_cols}
    out_df_cols[id_col] = ['']
    out_df = pl.DataFrame(out_df_cols)
    out_df = out_df.select(patients)
    chunk_size = int(MAX_FEATURE_NUMBER[type] / chunks + 1)
    for i in range(chunks):
        print(f'{dt.datetime.now()}: Start run {i} to filter the {type}')
        # load the data chunk by chunk as pandas dataframe
        agg_df = pd.DataFrame()
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
        
            if not os.path.isdir(file_path):
                df = pd.read_csv(file_path, sep='\t', index_col=0)
                df = df.iloc[i * chunk_size: (i + 1) * chunk_size, :]
                agg_df = df.join(agg_df, on=id_col, how='left')

        # perform a numpy calculation to convert the log odds data back to normalized counts
        # there does not exist any numpy interface with polars, such that a pandas dataframe is required
        agg_df = np.exp2(agg_df) - 1
        # convert the dataframe to a polars dataframe and proceed
        agg_index = pl.from_pandas(agg_df.index)
        agg_df = pl.from_pandas(agg_df)
        agg_df = agg_df.insert_column(0, agg_index)

        print(f'{dt.datetime.now()}: The aggregated DataFrame has shape {agg_df.shape}')
        filtered_df = remove_missing_values_threshold(agg_df, 0.05)
        out_df = out_df.extend(filtered_df.select(patients))

    print(f'{dt.datetime.now()}: Writing the filtered and aggregated file with dimension {out_df.shape}')
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    print(filtered_df.shape)
    filtered_df.write_csv(output_file_path, separator='\t')
    print(f'Finished writing data to file {output_file_path}')

In [25]:
def remove_missing_values_methylation():
    directory = f'UCSC_TCGA_Data_Clean/DNA_Methylation'
    output_directory = f'UCSC_TCGA_Data_Clean/Aggregated'
    output_file_path = os.path.join(output_directory, 'DNA_Methylation.tsv')
    patients = ['Probe']
    str_cols = {'Probe'}
    patients.extend(get_all_patients('DNA_Methylation'))
    out_df_cols = {col: [0.0] for col in patients if col not in str_cols}
    out_df_cols['Probe'] = ['']
    out_df = pl.DataFrame(out_df_cols)
    out_df = out_df.select(patients)
    # write the header to the output file
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    out_df.write_csv(output_file_path, separator='\t')
    # define the number of chunks to reduce the amount of used RAM
    chunks = 32
    chunk_size = int(len(probes_to_keep) / chunks + 1)
    for i in range(chunks):
        print(f'{dt.datetime.now()}: Start run {i} to filter the DNA methylation sides')
        agg_df = pl.DataFrame({'Probe': ['']})
        for file in os.listdir(directory):
            # output_file_path = os.path.join(output_directory, file.split('.')[0] + '_clean.tsv')
            file_path = os.path.join(directory, file)
        
            if not os.path.isdir(file_path):
                df = pl.read_csv(file_path, separator='\t', null_values=['NA']).slice(i * chunk_size, chunk_size)
                agg_df = df.join(agg_df, on='Probe', how='left')

        
        print(f'{dt.datetime.now()}: The aggregated DataFrame has shape {agg_df.shape}')
        # filtered_450_df = remove_missing_values_threshold(agg_df.drop(illumina_27_patients), 0.05)
        # filtered_27_df = remove_missing_values_threshold(agg_df.select(['Probe'] + illumina_27_patients), 0.05)
        # filtered_df = filtered_27_df.join(filtered_450_df, on='Probe', how='full', coalesce=True)
        filtered_df = remove_missing_values_threshold(agg_df.drop(illumina_27_patients), 0.05)
        out_df = filtered_df.select(patients)
        with open(output_file_path, mode='a') as f:
            print(f'{dt.datetime.now()}: Writing the filtered data of chunk {i} with dimension {out_df.shape}')
            out_df.write_csv(f, separator='\t', include_header=False)
            print('Data has been appended')

    print(f'Finished writing data to file {output_file_path}')

In [26]:
remove_missing_and_zero_expression('miRNA_Expression', 'miRNA_ID', 1)
remove_missing_and_zero_expression('lncRNA_Expression', 'Gene', 1)
remove_missing_and_zero_expression('Gene_Expression', 'Gene', 1)

2026-01-15 12:28:52.102654: Reading all patient ids
2026-01-15 12:28:52.977058: All patient ids read, in total 8494 patients
2026-01-15 12:28:53.056492: Start run 0 to filter the miRNA_Expression
2026-01-15 12:28:56.466318: The aggregated DataFrame has shape (1881, 8495)
(932, 8495)
2026-01-15 12:28:58.560536: Writing the filtered and aggregated file with dimension (933, 8495)
(932, 8495)
Finished writing data to file UCSC_TCGA_Data_Clean/Aggregated/miRNA_Expression.tsv
2026-01-15 12:28:59.203344: Reading all patient ids
2026-01-15 12:29:00.711052: All patient ids read, in total 8494 patients
2026-01-15 12:29:00.780316: Start run 0 to filter the lncRNA_Expression
2026-01-15 12:29:25.086301: The aggregated DataFrame has shape (15775, 8495)
(7450, 8495)
2026-01-15 12:29:41.127061: Writing the filtered and aggregated file with dimension (7451, 8495)
(7450, 8495)
Finished writing data to file UCSC_TCGA_Data_Clean/Aggregated/lncRNA_Expression.tsv
2026-01-15 12:29:43.655315: Reading all pati

In [27]:
def join_separate_methylation_protocols():
    # the protocols are processed separately to strictly filter the data
    # As a lot of patient data processed with Illumina 27k protocol contains 450.000 zeros, the variance is more likely to have a higher variance
    # To avoid filtering just because the protocol it is filtered by two different protocols, the filtering procedure is done seperately
    print(f'Reading and joining both DNA Methylation files for project {project}')
    df_27 = pl.read_csv(file_path_27, separator='\t')
    df_450 = pl.read_csv(file_path_450, separator='\t')
    
    df = df_27.join(df_450, on='Composite Element REF', how='full', coalesce=True)
    print(f'Files joined from shapes {df_27.shape} and {df_450.shape} to shape {df.shape}')
        
    df.write_csv(output_file, separator='\t')
    print(f'Methylation files joined into {output_file}')

In [28]:
remove_missing_values_methylation()

2026-01-15 12:32:41.459765: Reading all patient ids
2026-01-15 12:33:02.058517: All patient ids read, in total 8494 patients
2026-01-15 12:33:02.152039: Start run 0 to filter the DNA methylation sides
2026-01-15 12:33:31.601682: The aggregated DataFrame has shape (11395, 8495)
(752, 8495)
2026-01-15 12:33:45.916020: Writing the filtered data of chunk 0 with dimension (752, 8495)
Data has been appended
2026-01-15 12:33:46.415193: Start run 1 to filter the DNA methylation sides
2026-01-15 12:34:13.591039: The aggregated DataFrame has shape (11395, 8495)
(748, 8495)
2026-01-15 12:34:28.028348: Writing the filtered data of chunk 1 with dimension (748, 8495)
Data has been appended
2026-01-15 12:34:28.470694: Start run 2 to filter the DNA methylation sides
2026-01-15 12:34:50.992502: The aggregated DataFrame has shape (11395, 8495)
(771, 8495)
2026-01-15 12:35:05.831221: Writing the filtered data of chunk 2 with dimension (771, 8495)
Data has been appended
2026-01-15 12:35:06.222218: Start r

# Remove Unneeded Files

In [29]:
def remove_files_in_directory(dir):
    for file in os.listdir(dir):
        file_path = os.path.join(dir, file)
        os.remove(file_path)

In [30]:
## Free up device space
directories = [
    'UCSC_TCGA_Data/Gene_Expression',
    'UCSC_TCGA_Data/DNA_Methylation',
    # 'UCSC_TCGA_Data/DNA_Methylation_27',
    'UCSC_TCGA_Data/DNA_Methylation_450',
    'UCSC_TCGA_Data/miRNA_Data',
]

for dir in directories:
    remove_files_in_directory(dir)

## Create Clinical Data

In [ ]:
for project in TCGA_PROJECTS:
    download_tcga_data(project, "clinical")

In [ ]:
def create_clinical_data_df():
    directory = 'UCSC_TCGA_Data/Clinical_Data/'
    output_file_path = 'UCSC_TCGA_Data_Clean/clinical.tsv'
    barcode_project_list = []
    survival_time_column = ['days_to_last_follow_up.diagnoses', 'days_to_death.demographic']
    if len(os.listdir('UCSC_TCGA_Data_Clean/')) > 0:
        tcga_barcodes = set(pl.read_csv('UCSC_TCGA_Data_Clean/Aggregated/miRNA_Expression.tsv', separator='\t').columns[1:])
    
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        df = pl.read_csv(file_path, separator='\t', schema_overrides={'submitter_id.annotations': pl.Utf8()},ignore_errors=True)

        selected_columns = df.select(['sample', 'project_id.project', 'tissue_type.samples', 'vital_status.demographic',
                                      'days_to_death.demographic', 'days_to_last_follow_up.diagnoses'])

        for row in selected_columns.iter_rows(named=True):
            sample_code = str(row['sample'])
            project_id = str(row['project_id.project']).split('-')[1]
            sample_type = str(row['tissue_type.samples'])
            vital_status = int(str(row['vital_status.demographic']).lower() == 'dead')
            days_to_event = row[survival_time_column[vital_status]]
            days_to_event = int(days_to_event) if days_to_event is not None else 0
            if sample_type == 'Normal':
                project_id = 'NORMAL'

            if sample_code not in tcga_barcodes:
                continue
            barcode_project_list.append((sample_code, project_id, vital_status, days_to_event))

    barcode_project_df = pl.DataFrame(barcode_project_list, schema=["Barcode", "Tumor", 'Vital_Status', 'Days_to_event'], orient='row')
    barcode_project_df = barcode_project_df.with_columns(
        pl.col('Tumor').cast(pl.Categorical).to_physical().alias('Tumor_codes')
    )
    
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    barcode_project_df.write_csv(output_file_path, separator='\t')
    return barcode_project_df.shape

In [ ]:
df_shape = create_clinical_data_df()
print(df_shape)